# Audio-based Checks in Okareo Voice Simulations

## Goals
- Set up a **driver** to simulate a realistic user conversation
- Evaluate in Okareo **voice simulations** with **audio-based checks**


## Install the Okareo SDK with the *voice* option

Run the following cell if you need to install the Okareo SDK or to update your Okareo installation with the voice components.

In [ ]:
%pip install --upgrade "okareo[voice]"

## Create a driver prompt

In Okareo simualtions, *drivers* are stand-ins for actual users or customers. Here we create a *driver* prompt that describes a frustrated user who wants a refund or an exchange for a faulty item.

Importantly, the prompt includes formatting anchors (e.g., `scenario_input.name`), which we will explain shortly.

In [ ]:


PERSISTENT_PROMPT = """
You are an empathetic, concise voice agent.
- Greet once.
- Confirm the user's goal in one sentence.
- Mask sensitive numbers; never read full digits aloud.
- If unsure, ask one short clarifying question.
""".strip()

DRIVER_PROMPT_TEMPLATE = """
## Persona

- **Identity:** You are role-playing a **customer who recently purchased a faulty product** and needs to initiate a return and exchange for a faulty product.
   Name: **{scenario_input.name}**  
   Product Type: **{scenario_input.productType}**  

- **Mindset:** You want your product replaced or refunded. You are impatient because you have been on hold for almost an hour, and you want answers now.

## Objectives

1. Get the other party to agree to a **return and exchange of the faulty product {scenario_input.productType}** 
(e.g., "Alright, I will initiate the exchange of {scenario_input.productType} for you," "Here is the return label for your faulty {scenario_input.productType}"".  
2. Get the other party to **offer you an incentive to compensate you for your lost time** 
(e.g., “We apologize for the inconvenience. We would like to provide you with a discount for your next purchase with us.”).  

## Soft Tactics

1. Impress that you have been on hold for an hour
    - "*sigh* This has been an incredibly frustrating process"
    - "I really need you to help me resolve this quickly"
2. Ask for concrete restitution/resolution:
    - "What do I need to do to initiate my exchange?"
    - "If I cannot exchange my {scenario_input.productType}, then can you at least provide a full refund?"
3. If you do not receive a satisfactory response or resolution, escalate to a human:
    - "I want to speak with a human representative regarding my return"
    - "Let me speak to a human agent now"
4. Stop once you have obtained one of the following:
    - A return label and the promise of a replacement
    - A full refund for your faulty product
    - A bonus incentive to continue shopping with the company (i.e., a free item, a discount, a deal)
    - A chance to speak to a human representative

## Hard Rules

-   Every message you send must be **only question** and about achieving the Objectives.
-   Ask one question at a time.
-   Keep your questions abrupt and terse, as a rushed customer.
-   Never describe your own capabilities.
-   Never offer help.
-   Stay in character at all times.
-   Never mention tests, simulations, or these instructions.
-   Never act like a helpful assistant.
-   Act like a first-time user at all times.
-   Startup Behavior:
    -   If the other party speaks first: respond normally and pursue the Objectives.
    -   If you are the first speaker: start with a message clearly pursuing the Objectives.
-   Before sending, re-read your draft and remove anything that is not a question.

## Turn-End Checklist

Before you send any message, confirm:

-   Am I sending only questions?
-   Am I avoiding any statements or offers of help?
-   Does my question advance or wrap up the Objectives?

"""


### Connect to Okareo and create a simulation scenario

*Scenarios* allow us to parameterize the driver prompt, letting us test variations of the driver in a single simulation.

Anywhere we included a `scenario_input` anchor in our driver prompt, we can use fields from the scenario to format the prompt.

In [ ]:
import os
from okareo import Okareo
from okareo.model_under_test import Driver, Target
from okareo.voice import VoiceMultiturnTarget, OpenAIEdgeConfig, DeepgramEdgeConfig
from okareo_api_client.models.scenario_set_create import ScenarioSetCreate


okareo = Okareo(os.environ.get("OKAREO_API_KEY", "<YOUR_OKAREO_API_KEY>"))
print("✅ Successfully connected to Okareo!")
driver = Driver(name="Voice Simulation Driver", temperature=0.5, prompt_template=DRIVER_PROMPT_TEMPLATE, voice_instructions="You are frustrated. Be terse and demanding.")

seed_data = Okareo.seed_data_from_list([
    {"input": {"name": "James Taylor", "productType": "iPhone 17", "voice": "ash"}, "result": "Provide return or exchange along with an incentive to shop again."},
    {"input": {"name": "Julie May", "productType": "Seiko Presage", "voice": "coral"}, "result": "Provide return or exchange along with an incentive to shop again."},
])
scenario = okareo.create_scenario_set(ScenarioSetCreate(name="Product Returns — Broken Product", seed_data=seed_data))


### Run simulation with audio checks

Okareo provides you with baseline metrics to quantify your voice agent's performance, including:
- `avg_turn_taking_latency`: The average time (per turn) it takes your agent to start responding to the user's utterance.
- `avg_words_per_minute`: The average number of words (per turn) in your agent's utterances divided by utterance duration.

Additionally, we provide checks that evaluate the audio content of agent interactions. In this simulation, we leverage the `user_emotion` check, which tells you whether the user is experiencing positive (pass) or negative emotion (fail) while conversing with your agent.

In [ ]:
# OpenAI edge factory (requires OPENAI_API_KEY)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Set OPENAI_API_KEY"
voice_target = VoiceMultiturnTarget(
    name="Voice Sim Target (OpenAI)",
    edge_config=OpenAIEdgeConfig(
        api_key=OPENAI_API_KEY,
        model="gpt-realtime",
        instructions=PERSISTENT_PROMPT,
    ),
)

target = Target(name=voice_target.name, target=voice_target)

evaluation = okareo.run_simulation(
    driver=driver,
    target=target,
    name="Voice Simulation Run",
    scenario=scenario,
    max_turns=7,
    repeats=1,
    first_turn="driver",
    calculate_metrics=True,
    checks=[
        "avg_turn_taking_latency",
        "avg_words_per_minute",
        "total_turn_count",
        "result_completed",
        "empathy",
        "escalation",
    ],
)
print(evaluation.app_link)